In [6]:
import sys
print(sys.executable)
!pip install opencv-python


C:\Users\HP\encode\anantwave\Scripts\python.exe
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [7]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp
from math import ceil, log2

# === Setup ===
dataset_path = "C:/Users/HP/encode/anantwave/Multi_class/train"
categories = ["cat", "dog", "wild"]
num_images_per_category = 1000  # You can adjust this if needed

# === Feature Extraction using SIFT ===
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros(128)
    return descriptors.mean(axis=0)

# === Load Dataset in Parallel ===
def process_image(category, image_file):
    image_path = os.path.join(dataset_path, category, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is not None:
        feat = extract_features(image)
        return feat, category
    return None, None

all_features, labels = [], []
for category in categories:
    print(f"Processing {category} images...")
    path = os.path.join(dataset_path, category)
    image_files = [f for f in os.listdir(path) if f.endswith(".jpg")][:num_images_per_category]

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda f: process_image(category, f), image_files), total=len(image_files)))

    for feat, label in results:
        if feat is not None:
            all_features.append(feat)
            labels.append(label)

# === Standardization + PCA ===
scaler = StandardScaler()
scaled = scaler.fit_transform(all_features)

pca = PCA(0.95)
reduced = pca.fit_transform(scaled)

# === Quantum Configuration ===
original_dim = reduced.shape[1]
n_qubits = int(ceil(log2(original_dim)))
dim = 2**n_qubits

def pad(vec, target_dim):
    if len(vec) < target_dim:
        return np.pad(vec, (0, target_dim - len(vec)))
    else:
        return vec[:target_dim]

dev = qml.device("lightning.qubit", wires=n_qubits)

def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i+1)%len(wires)]])
    for i in range(len(wires)):
        qml.RX(gamma, wires=i)
        qml.RZ(beta, wires=i)

@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    qml.AmplitudeEmbedding(x / np.linalg.norm(x), wires=range(n_qubits), normalize=True, pad_with=0.0)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# === Encode All with Quantum Circuit ===
quantum_outputs = []
print("Quantum encoding in progress...")
for vec in tqdm(reduced):
    vec_padded = pad(vec, dim)
    vec_pnp = pnp.array(vec_padded, requires_grad=False)
    result = quantum_model(vec_pnp)
    quantum_outputs.append(result)

# === Save Output ===
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}
with open("quantum_AFHQ_encoded.pkl", "wb") as f:
    pickle.dump(final_data, f)

print("Quantum encoding complete for Cat, Dog, and Wildlife classes with padded Amplitude Embedding and QAOA layer.")


Processing cat images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:16<00:00, 13.15it/s]


Processing dog images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:55<00:00, 17.90it/s]


Processing wild images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:50<00:00, 19.68it/s]


Quantum encoding in progress...


100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:14<00:00, 208.69it/s]


Quantum encoding complete for Cat, Dog, and Wildlife classes with padded Amplitude Embedding and QAOA layer.


In [9]:
import pickle

# Load the quantum encoded data
with open("quantum_AFHQ_encoded.pkl", "rb") as f:
    data = pickle.load(f)
    
# Print the first 5 quantum encoded vectors and their labels
print("Quantum Encoded Outputs (First 5):")
for i in range(5):
    print(f"Sample {i+1}:")
    print("  Label    :", data["labels"][i])
    print("  Encoding :", data["quantum_encoded"][i])
    print()


Quantum Encoded Outputs (First 5):
Sample 1:
  Label    : cat
  Encoding : [array(0.8701905), array(0.73895513), array(0.77356545), array(0.62696102), array(-0.26144835)]

Sample 2:
  Label    : cat
  Encoding : [array(0.83470984), array(0.75914326), array(0.8091485), array(0.70800695), array(0.73816036)]

Sample 3:
  Label    : cat
  Encoding : [array(0.86190561), array(0.74723801), array(0.72188804), array(0.38755196), array(0.51138963)]

Sample 4:
  Label    : cat
  Encoding : [array(0.82555715), array(0.71952171), array(0.78968455), array(0.70965053), array(0.72570918)]

Sample 5:
  Label    : cat
  Encoding : [array(0.85667816), array(0.81378363), array(0.73935792), array(0.73883707), array(0.79031264)]



In [11]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp

# === Setup ===
dataset_path = "C:/Users/HP/encode/anantwave/Multi_class/train"  # Update to AFHQ dataset path
categories = ["cat", "dog", "wild"]  # Classes: Cat, Dog, Wildlife
num_images_per_category = 1000
all_features = []
labels = []

# === SIFT Feature Extraction ===
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros((128,))
    return descriptors.mean(axis=0)

# === Image Processing Loop ===
for category in categories:
    path = os.path.join(dataset_path, category)  # Paths for different categories
    image_files = [f for f in os.listdir(path) if f.endswith(".jpg")][:num_images_per_category]
    print(f"Encoding {category} images...")

    for image_file in tqdm(image_files):
        try:
            image_path = os.path.join(path, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            features = extract_features(image)
            all_features.append(features)
            labels.append(category)
        except Exception as e:
            print(f"Error processing {image_file}: {e}")


Encoding cat images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:04<00:00,  8.04it/s]


Encoding dog images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:57<00:00,  8.48it/s]


Encoding wild images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:12<00:00,  7.55it/s]


In [13]:
import pickle
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# === Standardization + PCA ===
scaler = StandardScaler()
scaled_features = scaler.fit_transform(all_features)

# Dynamic number of PCA components (limit to 8)
n_components = min(8, scaled_features.shape[1])
pca = PCA(n_components=n_components)
reduced_features = pca.fit_transform(scaled_features)

# === Quantum Configuration ===
n_qubits = n_components
dev = qml.device("default.qubit", wires=n_qubits)

# QAOA Layer
def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i + 1) % len(wires)]])
    for i in range(len(wires)):
        qml.RX(2 * gamma, wires=i)
        qml.RZ(2 * beta, wires=i)

# QNode
@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    qml.AmplitudeEmbedding(x / np.linalg.norm(x), wires=range(n_qubits), normalize=True, pad_with=0.0)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# === Quantum Encoding ===
quantum_outputs = []
for vec in reduced_features:
    vec_pnp = pnp.array(vec, requires_grad=False)
    result = quantum_model(vec_pnp)
    quantum_outputs.append(result)

# === Save Output ===
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}

with open("quantum_real_time_data.pkl", "wb") as f:
    pickle.dump(final_data, f)

print("Pipeline complete with adaptive qubit encoding and no resizing.")


Pipeline complete with adaptive qubit encoding and no resizing.


In [14]:
# Printing Head
print("\nPreview of Encoded Quantum Features:")
for i in range(min(5, len(quantum_outputs))):  # Ensure we don't exceed the length of the list
    print(f"Label: {labels[i]}, Quantum Output: {quantum_outputs[i]}")



Preview of Encoded Quantum Features:
Label: cat, Quantum Output: [tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.51012803, requires_grad=True), tensor(0.41513213, requires_grad=True), tensor(-0.18260537, requires_grad=True)]
Label: cat, Quantum Output: [tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.52362723, requires_grad=True), tensor(0.49620602, requires_grad=True), tensor(0.51134007, requires_grad=True)]
Label: cat, Quantum Output: [tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.54030231, requires_grad=True), tensor(0.4839308, requires_grad=Tr